In [ ]:
import os
os.environ['VLLM_USE_V1'] = '0'
from indextts.infer_v2_vllm import IndexTTS2
# from indextts.infer_v2 import IndexTTS2
import asyncio
index_tts = IndexTTS2(cfg_path="checkpoints/config.yaml", model_dir="checkpoints", use_cuda_kernel=False, gpu_memory_utilization=0.25)

In [ ]:

import importlib
import types
from pydub import AudioSegment
from indextts import infer_v2_vllm
from indextts.s2mel.modules import diffusion_transformer
from indextts.s2mel.modules import flow_matching
from indextts.utils import front
importlib.reload(infer_v2_vllm)
importlib.reload(diffusion_transformer)
importlib.reload(flow_matching)
importlib.reload(front)
from indextts.infer_v2_vllm import IndexTTS2
from indextts.s2mel.modules.flow_matching import CFM
from indextts.s2mel.modules.diffusion_transformer import DiT
import time

index_tts.infer_stream = types.MethodType(IndexTTS2.infer_stream, index_tts)
index_tts.infer_stream_optimized = types.MethodType(IndexTTS2.infer_stream_optimized, index_tts)
index_tts.infer_parallel = types.MethodType(IndexTTS2.infer_parallel, index_tts)
index_tts.tokenizer.split_segments = types.MethodType(front.TextTokenizer.split_segments, index_tts.tokenizer)
# index_tts.front = types.MethodType(front.Front.load, index_tts.front)
index_tts.s2mel.models['cfm'].solve_euler = types.MethodType(CFM.solve_euler, index_tts.s2mel.models['cfm'])
index_tts.s2mel.models['cfm'].estimator.forward = types.MethodType(DiT.forward, index_tts.s2mel.models['cfm'].estimator)

count = 0
concat_segment = None
from io import BytesIO
start_time = time.time()
async for chunk in index_tts.infer_stream_optimized(spk_audio_prompt='/root/RichooAgent/data/Female 1.mp3', text="""Encouraging Consistent Chess Practice. Tom could benefit from more consistent practice to really solidify his understanding and skills in chess. This is a common area for growth, and with some focused effort, we'll see great progress. To help Tom get more accustomed to chess, we recommend establishing a fun, regular practice routine at home. Tom could try playing one or two quick games daily, either with family or on a platform like ChessKid. Solving a few chess puzzles each week through an app or workbook is also an excellent way to sharpen his mind. Aiming for around 2-4 hours of practice a week will build a strong foundation and keep the game enjoyable and engaging for him.""", output_path="test_parallel2.wav", verbose=False, max_text_tokens_per_segment=120, interval_silence=0, first_handle_num=1, crossfade_ms=100):
    # print(len(chunk))
    # pass
    end_time = time.time()
    print(f"Time taken to receive chunk: {end_time - start_time:.2f} seconds, size: {len(chunk)}")
    start_time = end_time

    # with open(f"test_stream_{count}.wav", "wb") as f:
    #     f.write(chunk)
    if concat_segment is None:
        # concat_segment = AudioSegment.from_file(BytesIO(chunk), format="mp3")
        concat_segment = AudioSegment.from_raw(BytesIO(chunk), sample_width=2, frame_rate=22050, channels=1)
    else:
        # concat_segment = concat_segment.append(AudioSegment.from_file(BytesIO(chunk), format="mp3"), crossfade=0)
        concat_segment = concat_segment.append(AudioSegment.from_raw(BytesIO(chunk), sample_width=2, frame_rate=22050, channels=1), crossfade=0)
    
concat_segment.export("test_stream.mp3", format="mp3")
# await index_tts.infer_parallel(spk_audio_prompt='data/test_real.wav', text="""Tom could benefit from more consistent practice to really solidify his understanding and skills in chess. This is a common area for growth, and with some focused effort, we'll see great progress. To help Tom get more accustomed to chess, we recommend establishing a fun, regular practice routine at home. Tom could try playing one or two quick games daily, either with family or on a platform like ChessKid. Solving a few chess puzzles each week through an app or workbook is also an excellent way to sharpen his mind. Aiming for around 2-4 hours of practice a week will build a strong foundation and keep the game enjoyable and engaging for him.""", output_path="test_parallel2.wav", verbose=False, max_text_tokens_per_segment=120, interval_silence=0)


In [ ]:
from pydub import AudioSegment

# 加载两个音频文件
audio1 = AudioSegment.from_mp3("test_stream_1.wav")
audio2 = AudioSegment.from_mp3("test_stream_2.wav")

merged_chunk = audio1.append(audio2, crossfade=0)
merged_chunk.export("merged_chunk.mp3", format="mp3")
# def stream_with_crossfade(audio1, audio2, crossfade_ms=250):
#     # Step 1: 播放 audio1 主体部分（除去最后 crossfade_ms）
#     main_part = audio1[:-crossfade_ms]
#     yield main_part.raw_data  # 或导出为临时文件/分块传输

#     # Step 2: 交叉淡出淡入部分
#     fade_out = audio1[-crossfade_ms:].fade_out(crossfade_ms)
#     fade_in = audio2[:crossfade_ms].fade_in(crossfade_ms)
#     crossfaded = fade_out.overlay(fade_in)  # 混合
#     yield crossfaded.raw_data

#     # Step 3: 播放 audio2 剩余部分
#     remaining = audio2[crossfade_ms:]
#     yield remaining.raw_data
# crossfade_ms = 250
# a1 = audio1[:-crossfade_ms]
# a2 = audio2[crossfade_ms:]
# fade_out = audio1[-crossfade_ms:].fade_out(crossfade_ms)
# fade_in = audio2[:crossfade_ms].fade_in(crossfade_ms)
# crossfaded = fade_out.overlay(fade_in)

# merged_chunk = a1.append(crossfaded, crossfade=0)
# merged_chunk = merged_chunk.append(a2, crossfade=0)
# merged_chunk.export("merged_chunk.mp3", format="mp3")
# with open("merged_chunk.mp3", "wb") as f:
    # f.write(a1 + crossfaded + a2)


In [ ]:
import httpx
import requests
from pydub import AudioSegment
from io import BytesIO
import time
payload = {
    "prompt_audio_url": "https://beezstorageblob.blob.core.windows.net/emailblob/cloned_voice_0e3a2d2d-7a24-41d2-b1f5-c7af7f3ab284.wav",
    "text": """```title\n## Encouraging Consistent Chess Practice\n```\n```report\nTom could benefit from more consistent practice to really solidify his understanding and skills in chess. This is a common area for growth, and with some focused effort, we'll see great progress.\n\nTo help Tom get more accustomed to chess, we recommend establishing a fun, regular practice routine at home. Tom could try playing one or two quick games daily, either with family or on a platform like ChessKid. Solving a few chess puzzles each week through an app or workbook is also an excellent way to sharpen his mind. Aiming for around 2-4 hours of practice a week will build a strong foundation and keep the game enjoyable and engaging for him.\n```""",
}

concat_segment = None




try:
    with httpx.stream("POST", "http://127.0.0.1:62343/generate-speech-stream/", json=payload, follow_redirects=True) as r:
        # r.raise_for_status() # Ensure the request was successful
        start_time = time.time()
        for chunk in r.iter_bytes():
            end_time = time.time()
            print(f"Time taken to receive chunk: {end_time - start_time:.2f} seconds, size: {len(chunk)}")
            start_time = end_time
            if concat_segment is None:
                concat_segment = AudioSegment.from_file(BytesIO(chunk))
            else:
                concat_segment = concat_segment.append(AudioSegment.from_file(BytesIO(chunk)), crossfade=0)
            break
    concat_segment.export("test_stream.mp3", format="mp3")
except httpx.HTTPError as e:
    print(f"An HTTP error occurred: {e}")

In [ ]:
# transcribe base64
import base64
import requests
# with open("test_stream.mp3", "rb") as f:
with open("test_stream.mp3", "rb") as f:
    data = f.read()
    base64_data = base64.b64encode(data).decode("utf-8")
payload = {
    "audio_data": base64_data,
    "file_extension": "wav",
    "language": "en"
}

response = requests.post("http://127.0.0.1:62343/transcribe_base64/", json=payload)
print(response.json())

In [3]:
import httpx
from io import BytesIO
import time
from pydub import AudioSegment
async def generate_speech_stream(prompt_audio_url, text):
    # 1. regenerate speech
    # 2. return speech url
    try:
        payload = {
            "prompt_audio_url": prompt_audio_url,
            "text": text,
            "crossfade_ms": 50,
            "interval_silence": 200
        }
        async with httpx.AsyncClient(follow_redirects=True) as client:
            async with client.stream(
                "POST",
                'http://voice.trustbeez.com/generate-speech-stream/',
                json=payload,
                follow_redirects=True,
            ) as response:
                async for chunk in response.aiter_bytes(chunk_size=16000):
                    yield chunk
    except Exception as e:
        print(e)
    
prompt_audio_url = "https://beezstorageblob.blob.core.windows.net/emailblob/cloned_voice_male1.wav"
text = """```title\n## Encouraging Consistent Chess Practice\n```\n```report\nTom could benefit from more consistent practice to really solidify his understanding and skills in chess. This is a common area for growth, and with some focused effort, we'll see great progress.\n\nTo help Tom get more accustomed to chess, we recommend establishing a fun, regular practice routine at home. Tom could try playing one or two quick games daily, either with family or on a platform like ChessKid. Solving a few chess puzzles each week through an app or workbook is also an excellent way to sharpen his mind. Aiming for around 2-4 hours of practice a week will build a strong foundation and keep the game enjoyable and engaging for him.\n```"""
concat_segment = None
full_audio_bytes = BytesIO()
start_time = time.time()
async for chunk in generate_speech_stream(prompt_audio_url, text):
    print(len(chunk), time.time() - start_time)
    full_audio_bytes.write(chunk)
full_audio_bytes.seek(0)
audio_segment = AudioSegment.from_raw(full_audio_bytes, sample_width=2, frame_rate=22050, channels=1)
audio_segment.export("test_stream.mp3", format="mp3")


16000 2.0537772178649902
16000 2.055497169494629
16000 2.108867883682251
16000 2.172163724899292
16000 2.1736738681793213
16000 2.20696759223938
16000 2.2150564193725586
16000 2.215974807739258
16000 6.629493951797485
16000 6.629841089248657
16000 6.630020618438721
16000 6.630141973495483
16000 6.653737545013428
16000 6.660209894180298
16000 6.660399913787842
16000 6.675412893295288
16000 6.681427001953125
16000 6.685654640197754
16000 6.7090020179748535
16000 6.717679023742676
16000 6.722822904586792
16000 6.730906248092651
16000 6.757320880889893
16000 6.7613139152526855
16000 6.769680500030518
16000 6.785429239273071
16000 6.793383836746216
16000 6.802084922790527
16000 6.808416366577148
16000 6.827167749404907
16000 6.837985992431641
16000 6.852877140045166
16000 6.853297710418701
16000 6.861295223236084
16000 7.493634223937988
16000 7.5051188468933105
16000 7.518354654312134
16000 7.533416271209717
16000 7.535289287567139
16000 7.539444446563721
16000 7.572607755661011
16000 7.581

<_io.BufferedRandom name='test_stream.mp3'>

In [ ]:
import requests
start_time = time.time()
payload = {
    "prompt_audio_url": "https://beezstorageblob.blob.core.windows.net/emailblob/cloned_voice_female1.wav",
    "text": """```title\n## Encouraging Consistent Chess Practice\n```\n```report\nTom could benefit from more consistent practice to really solidify his understanding and skills in chess. This is a common area for growth, and with some focused effort, we'll see great progress.\n\nTo help Tom get more accustomed to chess, we recommend establishing a fun, regular practice routine at home. Tom could try playing one or two quick games daily, either with family or on a platform like ChessKid. Solving a few chess puzzles each week through an app or workbook is also an excellent way to sharpen his mind. Aiming for around 2-4 hours of practice a week will build a strong foundation and keep the game enjoyable and engaging for him.\n```""",
}

response = requests.post("http://127.0.0.1:62343/generate-speech", json=payload)
print(response.json())
end_time = time.time()
print(f"Time taken to generate speech: {end_time - start_time:.2f} seconds")


In [ ]:
import librosa
import soundfile as sf
import noisereduce as nr
import numpy as np
from pydub import AudioSegment
from pydub.effects import normalize

def enhance_voice(audio_file_path, output_file_path, noise_reduction_strength=0.8):
    """
    对音频文件进行降噪和音量标准化，以增强人声。

    参数:
    audio_file_path (str): 输入音频文件的路径。
    output_file_path (str): 处理后要保存的音频文件路径。
    noise_reduction_strength (float): 降噪强度，介于 0 和 1 之间。
                                     值越低，降噪越温和，但可能保留更多人声细节。
                                     建议从 0.7 到 0.9 之间尝试。
    """
    try:
        # 1. 加载音频文件
        print("正在加载音频文件...")
        audio_data, sample_rate = librosa.load(audio_file_path, sr=None)

        # 2. 降噪处理
        print("正在进行降噪...")
        # 使用传入的参数来控制降噪强度
        reduced_noise_audio = nr.reduce_noise(y=audio_data, sr=sample_rate, prop_decrease=noise_reduction_strength)
        print("降噪完成。")

        # 3. 音频标准化 (提升音量)
        print("正在进行音量标准化...")
        # 将 NumPy 浮点数组转换为 pydub 所需的格式（16位整数）
        # librosa 加载的范围是 -1.0 到 1.0，我们需要将其映射到 -32768 到 32767
        int_audio_data = np.int16(reduced_noise_audio * 32767)

        # 创建一个 pydub 的 AudioSegment 对象
        # sample_width=2 表示 16位音频，channels=1 表示单声道
        # 如果你的音频是立体声，需要将 channels 设置为 2
        audio_segment = AudioSegment(
            int_audio_data.tobytes(),
            frame_rate=sample_rate,
            sample_width=2,  # 16-bit
            channels=1      # 假设为单声道
        )

        # 应用标准化效果
        # headroom 参数可以防止削波，默认是 0.1 dB
        normalized_segment = normalize(audio_segment)
        print("音量标准化完成。")


        # 4. 导出处理后的音频
        print(f"正在导出文件到: {output_file_path}")
        normalized_segment.export(output_file_path, format="wav")

        print("处理成功！")
        return True

    except Exception as e:
        print(f"处理音频文件时出错: {e}")
        return False

# --- 使用示例 ---
if __name__ == '__main__':
    # 输入你的音频文件路径
    input_audio_path = '/workspace/RichooAgent/third_party/index-tts/data/enhanced_voice_audio.wav'  # 请替换成你自己的音频文件路径
    # 定义输出文件的路径
    output_audio_path = '/workspace/RichooAgent/third_party/index-tts/data/enhanced_voice_audio.wav'

    # 调用函数处理音频
    # 你可以调整 noise_reduction_strength 这个值来找到最佳平衡点
    # - 值较低 (如 0.7): 降噪更温和，人声音量和细节保留得更好。
    # - 值较高 (如 0.9): 降噪更彻底，但可能会对人声造成更多影响。
    success = enhance_voice(
        audio_file_path=input_audio_path,
        output_file_path=output_audio_path,
        noise_reduction_strength=0.9
    )

    if success:
        print(f"\n任务完成！增强后的音频已保存为 '{output_audio_path}'。")

In [ ]:
import librosa
import noisereduce as nr
import numpy as np
from pydub import AudioSegment
import io
from pydub.effects import normalize
def reduce_noise_and_normalize_volume_from_bytes(
    audio_bytes: bytes,
    sample_rate: int = None,
    noise_reduction_strength: float = 0.9
) -> bytes:
    """
    对内存中的音频字节进行降噪和音量标准化，返回处理后的 WAV 字节。

    参数:
    audio_bytes (bytes): 原始音频字节（如从 base64 解码后的数据）
    sample_rate (int): 如果已知采样率，可传入；否则自动检测
    noise_reduction_strength (float): 降噪强度，0~1

    返回:
    bytes: 处理后的 WAV 音频字节
    """
    try:
        # 1. 从字节加载音频数据
        print("Loading audio from bytes...")
        audio_data, detected_sample_rate = librosa.load(io.BytesIO(audio_bytes), sr=sample_rate)

        sample_rate = sample_rate or detected_sample_rate

        # 2. 降噪
        print("Reducing noise...")
        reduced_noise_audio = nr.reduce_noise(
            y=audio_data,
            sr=sample_rate,
            prop_decrease=noise_reduction_strength
        )
        print("Noise reduction completed.")

        # 3. 音量标准化
        print("Normalizing volume...")
        # 转换为 16-bit PCM
        int_audio_data = np.int16(reduced_noise_audio * 32767)

        # 创建 AudioSegment
        audio_segment = AudioSegment(
            data=int_audio_data.tobytes(),
            sample_width=2,  # 16-bit
            frame_rate=sample_rate,
            channels=1       # 单声道，如需立体声需调整
        )

        normalized_segment = normalize(audio_segment, headroom=0.1)
        print("Volume normalization completed.")

        # 4. 导出为字节（WAV 格式）
        buffer = io.BytesIO()
        normalized_segment.export(buffer, format="wav")
        buffer.seek(0)
        processed_bytes = buffer.getvalue()

        print("Audio processing from bytes successful!")
        return processed_bytes

    except Exception as e:
        print(f"Error processing audio from bytes: {e}")
        raise  # 建议抛出异常，让上层捕获处理

with open("/workspace/RichooAgent/third_party/index-tts/data/test_real.wav", "rb") as f:
    data = f.read()
    processed_bytes = reduce_noise_and_normalize_volume_from_bytes(data)
    with open("/workspace/RichooAgent/third_party/index-tts/data/test_realaaa.wav", "wb") as f:
        f.write(processed_bytes)


Loading audio from bytes...
Error processing audio from bytes: name 'io' is not defined


NameError: name 'io' is not defined